텐서보드로부터 값 뽑아내는 작업, 근데 이제 여러 텐서보드 로그를 취합하는.

valid_AUROC 기준 최고성능인 epoch의 통계값들을 뽑아내면 된다.

In [19]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd

In [20]:
# 사전 변수 넣기
tensorboard_dirs_txt = '/home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/statistics/tensorboard_to_pandas/encoding_main_test.txt'
output_file_name = 'encoding_main_test.csv'

tags = ['valid_Loss','valid_Accuracy','valid_F1_micro','valid_F1_weighted', 'valid_AUROC_macro','valid_AUROC_weighted','valid_auprc']
usable_tags = ['valid_Accuracy','valid_F1_micro', 'valid_AUROC_macro', 'valid_auprc'] # 일반적인 경우
# usable_tags = ['valid_AUROC_macro'] # stride 보려면 이것만 있어도 될 듯

In [21]:
with open(tensorboard_dirs_txt, 'r', encoding='utf-8') as file:
    tensorboard_dirs = file.read().splitlines()

In [26]:
# 병합된 데이터 저장용 리스트
highest_rows = []

for dir_path in tensorboard_dirs: # 각 로그마다
    print(f"Processing TensorBoard log: {dir_path}")
    ea = event_accumulator.EventAccumulator(dir_path)
    ea.Reload() # 텐서보드 로그 뽑아서 변수에 넣기기
    
    data = {} # 수집된 데이터를 저장할 딕셔너리
    epoch_time_map = {}  # 에포크와 wall_time 매핑

    # 태그(메트릭) 별로 에포크와 값 빼내서 저장
    for tag in tags:
        if tag in ea.Tags()["scalars"]:  # 태그가 존재하는지 확인
            events = ea.Scalars(tag)
            data[tag] = {
                "epoch": [event.step for event in events],  # step을 epoch으로 사용
                "value": [event.value for event in events],
            }
            # epoch -> wall_time 매핑 (처음 태그에서만 추출)
            if not epoch_time_map:
                epoch_time_map = {event.step: event.wall_time for event in events}
        else:
            print(f"Tag '{tag}' not found in the TensorBoard logs.")

    # 태그별 데이터프레임 생성
    dataframes = {tag: pd.DataFrame(values) for tag, values in data.items()}

    # 단일 데이터프레임으로 병합 (Epoch 기준)
    merged_df = pd.DataFrame({"epoch": dataframes[tags[0]]["epoch"]})  # 첫 태그의 epoch 사용
    for tag in tags:
        if tag in dataframes:
            merged_df[tag] = dataframes[tag]["value"]

    # 필요한 태그로 필터링
    filtered_df = merged_df[[tag for tag in usable_tags if tag in merged_df.columns]] # 에포크도 보고 싶으니 추가

    # 가장 높은 AUROC 값을 갖는 행 추출
    if "valid_AUROC_macro" in filtered_df.columns:
        highest_row = filtered_df.loc[filtered_df['valid_AUROC_macro'].idxmax()].copy()  # 명시적 복사
        highest_row["source_dir"] = dir_path.split('/')[-1]  # 로그 출처 추가

        highest_rows.append(highest_row)

# 최종 병합
final_df = pd.DataFrame(highest_rows)

# 결과 확인
print("\nFinal DataFrame with Highest AUROC Rows:")
print(final_df)

Processing TensorBoard log: /home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/IF/test/tensorboard/SNN_MLP_ver6_poisson_2024-12-27-13-02-25_test1
Processing TensorBoard log: /home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/IF/test/tensorboard/SNN_MLP_ver6_poisson_2024-12-27-13-02-25_test2
Processing TensorBoard log: /home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/IF/test/tensorboard/SNN_MLP_ver6_poisson_2024-12-27-13-02-25_test3
Processing TensorBoard log: /home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/IF/test/tensorboard/SNN_MLP_ver6_poisson_2024-12-27-13-02-25_test4
Processing TensorBoard log: /home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/IF/test/tensorboard/SNN_MLP_ver6_poisson_2024-12-27-13-02-25_test5
Processing TensorBoard log: /home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/IF/test/tensorboard/SNN_MLP_ver6_burst_2024-12-27-13-02-29_test1
Processing TensorBoard log: /home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/IF/test/tensorboard/SNN_MLP_ver6_burst_2024-12-27-13-02-29_test2
Pro

In [28]:
# 저장하기
final_df.to_csv(output_file_name, index=False)